In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
#!/usr/bin/env python
# coding: utf-8


In [ ]:
#For data processing
import pandas as pd
import numpy as np
import datetime as dt

#For data visualization
import matplotlib.pyplot as plt
import seaborn as sns

#For models
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

# IMPORTING IMPORTANT LIBRARIES
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop, SGD, Adam
import preprocessing 
import yfinance as yf
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df =  pd.read_csv("data\\reliance_stock.csv")

In [ ]:
df.tail(5)

In [ ]:
look_back = 150
forward_days = 100
num_periods = 20

In [ ]:
df = df['Close']

In [ ]:

array = df.values.reshape(df.shape[0],1)
array[:5]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scl = MinMaxScaler()
array = scl.fit_transform(array)
array[:5]

In [ ]:
len(array)

In [ ]:
division = len(array) - num_periods*forward_days
division

In [ ]:
array_test = array[division-look_back:]
array_train = array[:division]

In [ ]:

#Get the data and splits in input X and output Y, by spliting in `n` past days as input X 
#and `m` coming days as Y.
def processData(data, look_back, forward_days,jump=1):
    X,Y = [],[]
    for i in range(0,len(data) -look_back -forward_days +1, jump):
        X.append(data[i:(i+look_back)])
        Y.append(data[(i+look_back):(i+look_back+forward_days)])
    return np.array(X),np.array(Y)

In [ ]:
X_test,y_test = processData(array_test,look_back,forward_days,forward_days)
y_test = np.array([list(a.ravel()) for a in y_test])

X,y = processData(array_train,look_back,forward_days)
y = np.array([list(a.ravel()) for a in y])

from sklearn.model_selection import train_test_split
X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
print(X_train.shape)
print(X_validate.shape)
print(X_test.shape)
print(y_train.shape)
print(y_validate.shape)
print(y_test.shape)

In [ ]:
model = Sequential()

model.add(LSTM(units=128, return_sequences=True, input_shape=(look_back,1)))
# model.add(Dense(units = 1))
model.add(Dropout(0.2))

model.add(LSTM(units=64, return_sequences=True))
# model.add(Dense(units = 1))
model.add(Dropout(0.2))

model.add(LSTM(units=32, return_sequences=False))
# # model.add(Dense(units = 1))
model.add(Dropout(0.2))

model.add(Dense(forward_days)) #output layer
# model.add(Activation('relu'))

import time #helper libraries
start = time.time()
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
print ('compilation time : ', time.time() - start)

In [ ]:
model.summary()

In [ ]:
# model.compile(optimizer = 'adam', loss = 'mean_squared_error')
history = model.fit(X_train, y_train, epochs = 100, batch_size = 32, validation_data=(X_validate,y_validate),shuffle=True, verbose =2)

In [ ]:
plt.figure(figsize = (15,10))

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.title("Model Train Loss v Validation Loss")
plt.show()

In [ ]:
Xt = model.predict(X_test)

In [ ]:
plt.figure(figsize = (15,10))

for i in range(0,len(Xt)):
    plt.plot([x + i*forward_days for x in range(len(Xt[i]))], scl.inverse_transform(Xt[i].reshape(-1,1)), color='r')
    
plt.plot(0, scl.inverse_transform(Xt[i].reshape(-1,1))[0], color='r', label='Prediction') #only to place the label
    
plt.plot(scl.inverse_transform(y_test.reshape(-1,1)), label='Target')

plt.legend(loc='best')

plt.ylabel('stock price value')
plt.xlabel('trading days')
plt.title("Predicttion v Target")
plt.show()
plt.show()

In [ ]:
division = len(array) - num_periods*forward_days

leftover = division%forward_days+1

array_test = array[division-look_back:]
array_train = array[leftover:division]

In [ ]:
Xtrain,ytrain = processData(array_train,look_back,forward_days,forward_days)
Xtest,ytest = processData(array_test,look_back,forward_days,forward_days)

In [ ]:

Xtrain = model.predict(Xtrain)
Xtrain = Xtrain.ravel()

Xtest = model.predict(Xtest)
Xtest = Xtest.ravel()

In [ ]:

y = np.concatenate((ytrain, ytest), axis=0)

In [ ]:
plt.figure(figsize = (15,10))

# Data in Train/Validation
plt.plot([x for x in range(look_back+leftover, len(Xtrain)+look_back+leftover)], scl.inverse_transform(Xtrain.reshape(-1,1)), color='r', label='Train')
# Data in Test
plt.plot([x for x in range(look_back +leftover+ len(Xtrain), len(Xtrain)+len(Xtest)+look_back+leftover)], scl.inverse_transform(Xtest.reshape(-1,1)), color='y', label='Test')

#Data used
plt.plot([x for x in range(look_back+leftover, look_back+leftover+len(Xtrain)+len(Xtest))], scl.inverse_transform(y.reshape(-1,1)), color='b', label='Actual')

#Initial data. It should overlap the data used
#plt.plot(scl.inverse_transform(array), color='b', label='Esperado')

plt.legend(loc='best')
plt.show()

In [ ]:
predicted_stock_price = scl.inverse_transform(Xtest.reshape(-1,1))


In [ ]:
target = scl.inverse_transform(y_test.reshape(-1,1))

In [ ]:
# Visualising the results

plt.figure(figsize=(20,10))
plt.plot(target, color = 'green', label = 'Stock Price')
plt.plot(predicted_stock_price, color = 'red', label = 'Predicted Price')
plt.title('Stock Price Prediction')
plt.xlabel('Trading Day')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score
from math import sqrt
mae = mean_absolute_error(target, predicted_stock_price)
print('MAE: %f' % mae)


mse = mean_squared_error(target, predicted_stock_price)
rmse = sqrt(mse)
print('RMSE: %f' % rmse)

r2 = r2_score(target, predicted_stock_price)
print('R2: %f' % r2)